# Appendix: An Interactive Console to Generate Movies Fulfilling Certain Requirements
Suppose you are a director, and you want to make a movie that fulfills certain genre, budget, and profit requirements. 
This small console performs lookup to find a movie that fulfills your requirements. 

I eventually hope to add a predictive element (so that one can predict a movie's profits based on self-generated features and using models trained in previous sections.)

In [75]:
import pandas as pd
import datetime as dt
import ast
import numpy as np
import math
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings(action='ignore')

In [76]:
data_path = "data/"
m = pd.read_csv(data_path + "movies_postprocessing.csv")
entities = {}
for col in ["genres", "keywords","plot_keywords","all_keywords" ,"production_companies", "production_countries", "spoken_languages"]:
    m[col] = pd.Series(ast.literal_eval(b) for b in m[col])
genres = ['Mystery', 'Romance', 'History', 'Family', 'Science Fiction', 'Horror', 'Crime', 'Drama', 'Fantasy', 'Animation', 'Music', 'Adventure', 'Foreign', 'Action', 'TV Movie', 'Comedy', 'Documentary', 'War', 'Thriller', 'Western']

### Interactive Console/ Movie Generator

In [77]:

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False


def get_closest_to_median(subset_, target, q_tile):
    subset_["diff_med"] = abs(subset_[target] - subset_[target].quantile(q_tile))
    return subset_.loc[subset_["diff_med"].idxmin()]
    

In [78]:
def run_console():
    perc = "default"
    allowed_genres = [a for a in genres if a not in  ['Documentary', 'Foreign']]
    allowed_targets = ['popularity', 'vote_average', 'imdb_score','movie_facebook_likes', 'profit']
    budget = 0
    genre = ""
    bl,bh = 0,0
    done = False

    while not done:
        genre = raw_input("\n Tell me your genre {}: ".format(str(allowed_genres)))
        if genre == "quit": done = True; break;
        if genre in allowed_genres: break
        print("Sorry, try again.")
    subset = m[m.genres.apply(lambda x: genre in x)]

    while not done:
        target = raw_input("\n Tell me what you're optimizing for {}: \n \
                             You can also add a percentile (0-100) after the category \n \
                           following the example 'profit 65' and we'll get you the film\n \
                           closest to that percentile for the target.\n \
                           Otherwise we'll serve you the default - the median!\n \
                           Please follow the format (lowercase and one space!)"
                           .format(str(allowed_targets)))   
        if target == "quit": done = True; break;
        target_ = target.strip().split(" ")
        target_ = [a for a in target_ if len(a) > 0]
        target = target_[0]
        perc = "default"
        print(target_)
        if len(target_) > 1:
            perc = target_[1]
        perc = 50 if not isfloat(perc) else float(perc)
        if not (perc > 0 and perc < 100): perc = 50
        perc = perc / 100.0
        if target in allowed_targets: break
        print("Sorry, try again.")

    while not done:
        repeat = False
        budget = raw_input("\n Tell me your budget [Range: 1e5 - 1e9, 10-90% percentiles for your genre are {0:.2e} to {1:.2e}]: \n \
                       You can also give a percentile range (0-100), in which case we will interpret it as a percentile.\n \
                       Either give one value or two; if you give two numbers we will interpret that \n \
                       as your full budget/percentile range, or if you give one, we will give you movies  \n \
                       close to that budget, if any exist. Finally, leave empty to get values over all movies. \n \
                       Please follow the format (one number or two numbers separated by one space!)"
                       .format(np.percentile(subset["budget"], 10), np.percentile(subset["budget"], 90)))  
        if budget == "quit": done = True; break;

        budget_ = budget.strip().split(" ")
        budget_ = [a for a in budget_ if len(a) > 0]
        if len(budget_) == 0:
            bl,bh = -1000, 100000000000
        elif len(budget_) == 1 and isfloat(budget_[0]):
            budget = float(budget_[0])
            if budget >0 and budget < 100:
                budget = np.percentile(subset["budget"], budget)
            if budget > np.min(subset["budget"]) and budget < np.max(subset["budget"]):
                bl,bh = budget/math.sqrt(10), budget*math.sqrt(10)
            else:
                repeat = True
        elif len(budget_)> 1 and isfloat(budget_[0]) and isfloat(budget_[1]) \
                                                and float(budget_[0]) < float(budget_[1]):
            bl,br = float(budget_[0]), float(budget_[1])
            if bl >0 and bl < 100: bl = np.percentile(subset["budget"], bl)
            if br >0 and br < 100: br = np.percentile(subset["budget"], br)
            if (bl > np.min(subset["budget"]) and bl < np.max(subset["budget"])) and \
            (br > np.min(subset["budget"]) and br < np.max(subset["budget"])) and bl < br:
                bl, bh = bl, br
            else: repeat = True
        else: 
            repeat = True

        if not repeat:
            subset_ = subset[(subset.budget > bl) & (subset.budget < bh) & (subset[target].notnull())]
            if len(subset)==0:
                print("Please pick another budget range. Unfortuantely, there were too few films in your selected genre with your budget range!\n")
                repeat= True
            else:
                s_med = get_closest_to_median(subset_, target, perc)
            if not repeat:
                break

        if not (budget > 1e5 and budget < 1e9): 
            print("Sorry, try again [Range: 1e5 - 1e9]")
        else:
            print("Something was wrong with your input. Please try again.")


    if not done:
    #     subset_ = subset[(subset.budget > bl) & (subset.budget < bh)]
    #     s_med = get_closest_to_median(subset_, target, perc)
    #     if len(s_med) == 0:
    #         bl,bh = budget/math.sqrt(100), budget*math.sqrt(100)
    #         subset_ = subset[(subset.budget > bl) & (subset.budget < bh)]
    #         s_med = get_closest_to_median(subset_, target, perc) #subset_[subset_[target]==subset_.median()[target]]
    #     if len(s_med) == 0:
    #         bl,bh = budget/math.sqrt(1000), budget*math.sqrt(1000)
    #         subset_ = subset[(subset.budget > bl) & (subset.budget < bh)]
    #         s_med = get_closest_to_median(subset_, target, perc) #subset_[subset_[target]==subset_.median()[target]]
    #     if len(s_med) == 0:
    #         print("Sorry, there were no films close to your budget range! Here is a reference film in your genre:\n")
    #         subset_ = subset
    #         s_med = get_closest_to_median(subset_, target, perc) #subset_[subset_[target]==subset_.median()[target]]

        s = s_med
        actors_list = [a for a in [s['actor_1_name'], s['actor_2_name'], s['actor_3_name']] if a is not None]

        print("\n\n\n"+
             "Your reference film is: {}\n".format(s["original_title"]) +
            "{} \n".format(s['overview']) +
            "\n" +
            "This film is a {} film from {},\n".format(int(s['title_year']),s['country']) +
            "has genres: {}\n".format(str(s['genres'])[1:-1]) +
            "and has budget {}.\n".format(s['budget']) +
            "It was directed by {} \n".format(s['director_name']) +
            "and stars {}. \n".format(str(actors_list)[1:-1]) +
            "\n" +
            "It's keywords are: {} \n".format(str(s['all_keywords'])[1:-1]) +
            "and it was produced by: {}.\n".format(str(s['production_companies'])[1:-1]) +
            "It's scores are: \n" +
            "    popularity: {}, vote_average: {},".format(s['popularity'],s['vote_average']) +
            "    imdb_score: {}, movie_facebook_likes: {}, profit: {} \n".format(s['imdb_score'],s['movie_facebook_likes'],s['profit']) +
            "    a value close to the {} percentile for your target {} value {} \n".format(int(perc*100), target,s[target]) +
            "for movies in your budget range: {0:.2e} to {1:.2e}".format(bl, bh)                                               
        )

In [79]:
run_console()


 Tell me your genre ['Mystery', 'Romance', 'History', 'Family', 'Science Fiction', 'Horror', 'Crime', 'Drama', 'Fantasy', 'Animation', 'Music', 'Adventure', 'Action', 'TV Movie', 'Comedy', 'War', 'Thriller', 'Western']:  Music

 Tell me what you're optimizing for ['popularity', 'vote_average', 'imdb_score', 'movie_facebook_likes', 'profit']: 
                              You can also add a percentile (0-100) after the category 
                            following the example 'profit 65' and we'll get you the film
                            closest to that percentile for the target.
                            Otherwise we'll serve you the default - the median!
                            Please follow the format (lowercase and one space!) vote_average


['vote_average']



 Tell me your budget [Range: 1e5 - 1e9, 10-90% percentiles for your genre are 3.98e+06 to 6.00e+07]: 
                        You can also give a percentile range (0-100), in which case we will interpret it as a percentile.
                        Either give one value or two; if you give two numbers we will interpret that 
                        as your full budget/percentile range, or if you give one, we will give you movies  
                        close to that budget, if any exist. Finally, leave empty to get values over all movies. 
                        Please follow the format (one number or two numbers separated by one space!) 60





Your reference film is: Hairspray
Pleasantly plump teenager, Tracy Turnblad and her best friend, Penny Pingleton audition to be on The Corny Collins Show – and Tracy wins. But when scheming Amber Von Tussle and her mother plot to destroy Tracy, it turns to chaos. 

This film is a 2007 film from USA,
has genres: 'Family', 'Comedy', 'Music', 'Romance'
and has budget 50000000.
It was directed by Adam Shankman 
and stars 'Jerry Stiller', 'Elijah Kelley', 'Paul Dooley'. 

It's keywords are: 'dance', '1960s', 'integration', 'tv show', 'music', 'performance', 'friend', 'best friend', 'based on film', 'duel', 'coloured', 'overweight woman', 'audition', 'tv dance show', 'school party', 'television', 'equality', 'based on stage musical', 'race politics', 'xenophobia', 'races', 'dream' 
and it was produced by: 'New Line Cinema', 'Ingenious Film Partners', 'Offspring Entertainment', 'Zadan / Meron Productions'.
It's scores are: 
    popularity: 31.160542, vote_average: 6.5,    imdb_score: 6.7, 